## Tensorの基礎

In [2]:
# ローカル環境の場合や、新規仮装環境などの場合は、pip install torchが必要
import torch

In [3]:
# tensorを作成
my_list = [1, 2, 3, 4]
tensor_from_list = torch.tensor(my_list)
print(tensor_from_list)

tensor([1, 2, 3, 4])


In [4]:
type(tensor_from_list)

torch.Tensor

In [5]:
# intのリストで作成したTensorのデータ形はint64
tensor_from_list.dtype

torch.int64

デフォルトの要素のデータ型は`float32`である。深層学習では、膨大なデータを扱うことが多いので、データ型を意識して作ることが重要になる。

In [6]:
# floatのlistで作ったtensorはfloat32
my_list = [1., 2., 3., 4.]
tensor_from_list = torch.tensor(my_list)
print(tensor_from_list)
print(tensor_from_list.dtype)

tensor([1., 2., 3., 4.])
torch.float32


dtype引数を使ってデータ型を指定することもできる。(例:dtype=torch.float64)

In [7]:
# dtype引数でデータ型を指定
tensor_from_list = torch.tensor(my_list, dtype=torch.float64)
print(tensor_from_list)
print(tensor_from_list.dtype)

tensor([1., 2., 3., 4.], dtype=torch.float64)
torch.float64


### 基本的な行列

In [8]:
self_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]]) # オリジナルの行列
zeros_tensor = torch.zeros((2, 3)) # 全要素が0の行列
ones_tensor = torch.ones(2, 3) # 全要素が1の行列
eye_tensor = torch.eye(3) # 単位行列
random_tensor = torch.rand(2, 3) # [0, 1)の一様分布

In [9]:
print(self_tensor)
print(zeros_tensor, zeros_tensor.dtype)
print(ones_tensor, ones_tensor.dtype)
print(eye_tensor, eye_tensor.dtype)
print(random_tensor, random_tensor.dtype)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[0., 0., 0.],
        [0., 0., 0.]]) torch.float32
tensor([[1., 1., 1.],
        [1., 1., 1.]]) torch.float32
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]) torch.float32
tensor([[0.5715, 0.2629, 0.3261],
        [0.4740, 0.2598, 0.1304]]) torch.float32


`.size()` を用いることで，Tensorのサイズを確認することができます。

`.size()`のエイリアスとして`.shape`も存在するので，numpyのように`.shape`で取得することも可能です。
引数を指定することで特定の次元のサイズのみを取得することも可能です。

In [10]:
# Tensorのshapeを表示
print(zeros_tensor.size()) # Tensorのサイズを取得
print(zeros_tensor.shape) # .shapeでも可能

print(zeros_tensor.size(0)) # Tensorの0次元目のサイズを取得

torch.Size([2, 3])
torch.Size([2, 3])
2


### Tensorの操作

In [11]:
tensor_example = torch.rand((2,3,4))
tensor_example.shape

torch.Size([2, 3, 4])

In [12]:
# 転置
permuted_tensor = torch.permute(tensor_example, (1, 0, 2)) # ランク（次元）の順番入れ替えを指定して転置
print("Original shape of tensor:", tensor_example.shape)
print("Permuted tensor:", permuted_tensor.shape)

# torch.transpose()では、2つの軸を入れ替えるだけ
transposed_tensor = torch.transpose(tensor_example, 0, 1) # 0次元と1次元を入れ替え
print("Original shape of tensor:", tensor_example.shape)
print("Transposed tensor:", transposed_tensor.shape)

Original shape of tensor: torch.Size([2, 3, 4])
Permuted tensor: torch.Size([3, 2, 4])
Original shape of tensor: torch.Size([2, 3, 4])
Transposed tensor: torch.Size([3, 2, 4])


In [13]:
# 2次元の転置では、.T/.t()を用いることで簡単に転置することができる（ランクを指定する必要がないから可能）
tensor_2rank_exampe = torch.rand((2, 3))
print("Original shape of tensor:", tensor_2rank_exampe.shape)
print("Transposed tensor:", (tensor_2rank_exampe.T).shape)
print("Transposed tensor:", (tensor_2rank_exampe.t()).shape)

Original shape of tensor: torch.Size([2, 3])
Transposed tensor: torch.Size([3, 2])
Transposed tensor: torch.Size([3, 2])


In [14]:
# reshape
tensor_example = torch.rand((2, 3, 4))
reshaped_tensor = tensor_example.reshape(6, 4)

# 結果を表示
print("Original shape of tensor:", tensor_example.shape)
print("Reshaped tensor:", reshaped_tensor.shape)

Original shape of tensor: torch.Size([2, 3, 4])
Reshaped tensor: torch.Size([6, 4])


In [15]:
# 基本的には元のメモリをそのまま使っている
# reshape後のtensorを変更すると，元のtensorも変更されている
print(tensor_example[0]) # reshape前の行列
reshaped_tensor[0] = 0 # reshape後の行列の0次元目のデータを全て0に変換する
print(tensor_example[0]) # reshape前の行列の再確認

tensor([[0.9446, 0.0781, 0.2183, 0.7480],
        [0.0774, 0.3435, 0.4626, 0.4066],
        [0.4649, 0.2989, 0.9534, 0.4040]])
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0774, 0.3435, 0.4626, 0.4066],
        [0.4649, 0.2989, 0.9534, 0.4040]])


Tensorでは、メモリを効率的に使用するために、reshapeなどをする際にコピーを作るのではなく、そのままのメモリを使用していることが多い。

しかし、不連続なメモリのデータに対しては、reshapeする際にコピーを作成するという特殊な点がある。

`.view`を用いることで、連続データのみの変換をすることができるため、メモリの効率化を重視するプロダクトを作成する際は使ってみるのも良い。

In [16]:
# メモリが連続でなければreshape後のtensorはコピーを返す
# 連続的なメモリレイアウトを持つTensorを作成
x = torch.tensor([[1, 2], [3, 4], [5, 6]])

# 連続的なメモリレイアウトを持たない部分Tensorを取得 (transposed tensor)
y = x.T

# y は非連続的なメモリレイアウトを持つことを確認
print(y.is_contiguous())  # False が出力される

# reshape を使用して y の形状を変更
z = y.reshape(-1)
# 以下はエラーになる
# z = y.view(-1)

# z が y とメモリを共有していないことを確認 (コピーが作成されたことを意味する)
print(z.data_ptr() == y.data_ptr())  # False が出力される

False
False


`torch.squeeze() `で多次元配列を1次元配列に変換する。

In [17]:
# flatten
flattened_tensor = torch.flatten(tensor_example)
print("Original shape of tensor:", tensor_example.shape)
print("Flattened tensor:", flattened_tensor.shape)

Original shape of tensor: torch.Size([2, 3, 4])
Flattened tensor: torch.Size([24])


`torch.squeeze()`メソッドを用いることで、tensorの形状から、サイズが１の次元を削除することができる。

In [20]:
# squeeze(サイズが1の次元を削除)
tensor_example = torch.tensor([[[1], [2], [3]]])
print(tensor_example.shape)
print((torch.squeeze(tensor_example)).shape)

torch.Size([1, 3, 1])
torch.Size([3])


`torch.unsqueeze`を使用することで、指定した位置に新しい次元を追加することができる。

In [21]:
# unsqueeze (指定した位置に新しい次元を追加)
unsqueeze_tensor = torch.unsqueeze(tensor_example, 0)
print(unsqueeze_tensor.shape)

torch.Size([1, 1, 3, 1])


In [22]:
# unsqueezeと同様の操作
tensor_example[None, :, :, :].shape

torch.Size([1, 1, 3, 1])

### Tensorの便利関数

In [25]:
tensor_example = torch.rand((2, 3))
print(tensor_example)

tensor([[0.1822, 0.5344, 0.7238],
        [0.0502, 0.7529, 0.9901]])


#### スカラー演算

numpyと同様にスカラー演算を実装することができます．

In [30]:
a = torch.rand((2, 3))
b = torch.rand((2, 3))

print("# Tensor1:")
print(a)
print("# Tensor2:")
print(b)

print("# 足し算:")
print(a + b)

print("# 引き算:")
print(a - b)

print("# 掛け算:")
print(a * b)

print("# 割り算:")
print(a / b)

print("# log:")
print(torch.log(a))

print("# exp:")
print(torch.exp(a))

print("# ルート:")
print(torch.sqrt(a))

# Tensor1:
tensor([[0.5831, 0.9767, 0.8351],
        [0.2472, 0.4399, 0.9462]])
# Tensor2:
tensor([[0.8924, 0.6374, 0.0056],
        [0.6555, 0.1406, 0.8447]])
# 足し算:
tensor([[1.4755, 1.6141, 0.8407],
        [0.9027, 0.5805, 1.7909]])
# 引き算:
tensor([[-0.3093,  0.3393,  0.8295],
        [-0.4083,  0.2994,  0.1015]])
# 掛け算:
tensor([[0.5203, 0.6225, 0.0047],
        [0.1621, 0.0618, 0.7993]])
# 割り算:
tensor([[  0.6534,   1.5323, 149.3204],
        [  0.3771,   3.1300,   1.1202]])
# log:
tensor([[-0.5394, -0.0236, -0.1802],
        [-1.3975, -0.8211, -0.0553]])
# exp:
tensor([[1.7916, 2.6556, 2.3050],
        [1.2805, 1.5526, 2.5760]])
# ルート:
tensor([[0.7636, 0.9883, 0.9138],
        [0.4972, 0.6633, 0.9727]])


#### 集約演算

集約演算もnumpyと同様に実装できますが，集約する次元をaxisではなくdimで指定することに注意する必要があります．

In [31]:
# サンプルのTensorを作成
tensor_example = torch.rand((2, 3))
print(tensor_example)

tensor([[0.2702, 0.1395, 0.1862],
        [0.6686, 0.5059, 0.1659]])


In [42]:
# 合計
sum_tensor = torch.sum(tensor_example)

# 平均
mean_tensor = torch.mean(tensor_example)

# 標準偏差
std_tensor = torch.std(tensor_example)

# 平方根
sqrt_tensor = torch.sqrt(tensor_example)

# 最大値
max_tensor = torch.max(tensor_example)

# 最小値
min_tensor = torch.min(tensor_example)

# 結果を表示
print("Sum:\n", sum_tensor, "\n")
print("Mean:\n", mean_tensor, "\n")
print("Standard Deviation:\n", std_tensor, "\n")
print("Square Root:\n", sqrt_tensor, "\n")
print("Maximum:\n", max_tensor, "\n")
print("Minimum:\n", min_tensor, "\n")

Sum:
 tensor(1.9363) 

Mean:
 tensor(0.3227) 

Standard Deviation:
 tensor(0.2157) 

Square Root:
 tensor([[0.5198, 0.3735, 0.4315],
        [0.8177, 0.7112, 0.4073]]) 

Maximum:
 tensor(0.6686) 

Minimum:
 tensor(0.1395) 



### 行列演算

#### 加減算及び要素ごとの乗除算

In [43]:
# 3✖️3のTensorを作成
a = torch.rand((3, 3))
b = torch.rand((3, 3))
print(a)
print(b)

tensor([[0.8942, 0.8538, 0.1573],
        [0.4117, 0.3880, 0.0316],
        [0.4430, 0.6755, 0.2923]])
tensor([[0.1635, 0.1268, 0.3612],
        [0.1728, 0.0137, 0.6081],
        [0.5384, 0.9433, 0.9546]])


In [45]:
# 加減算および要素毎の剰余算
add = a + b
sub = a - b
mul = a * b
div = a / b


print("加算:\n", add)
print("減算:\n", sub)
print("乗算:\n", mul)
print("剰余算:\n", div)

加算:
 tensor([[1.0578, 0.9806, 0.5185],
        [0.5845, 0.4017, 0.6397],
        [0.9814, 1.6189, 1.2469]])
減算:
 tensor([[ 0.7307,  0.7270, -0.2039],
        [ 0.2389,  0.3742, -0.5765],
        [-0.0954, -0.2678, -0.6623]])
乗算:
 tensor([[0.1462, 0.1083, 0.0568],
        [0.0711, 0.0053, 0.0192],
        [0.2385, 0.6373, 0.2791]])
剰余算:
 tensor([[ 5.4685,  6.7341,  0.4354],
        [ 2.3830, 28.2461,  0.0520],
        [ 0.8227,  0.7161,  0.3062]])


#### 行列の積

In [47]:
# 行列の積
mm = torch.mm(a, b)
matmul = torch.matmul(a, b)
at_op = a @ b

print(mm)
print(matmul)
print(at_op)

tensor([[0.3784, 0.2735, 0.9923],
        [0.1514, 0.0873, 0.4148],
        [0.3465, 0.3412, 0.8499]])
tensor([[0.3784, 0.2735, 0.9923],
        [0.1514, 0.0873, 0.4148],
        [0.3465, 0.3412, 0.8499]])
tensor([[0.3784, 0.2735, 0.9923],
        [0.1514, 0.0873, 0.4148],
        [0.3465, 0.3412, 0.8499]])


### ブロードキャスティング

* 演算する2つのtensorが同じ次元ではない場合，次元が少ない方が次元を増やすこ
とで演算を可能にする

 • Numpy同様，broad casDngの機能がある

 • 深層学習のコーディングでは頻出

 * ブロードキャスティングのルール

   • rank数が異なる場合，少ない方の配列のshapeの左側にサイズ1の次元を追加する(例: (2, 3) -> (1, 2, 3))

   • shapeの右側から値(サイズ数)を比較し，数が一致するか，サイズが1であればブロードキャスティングが可能

     • サイズ1の次元が，もう片方の次元のサイズに拡大する

In [48]:
# (3, 3)とスカラーの演算
a = torch.rand((3, 3))
scaler = 5
resulet1 = a + scaler
print(a)
print(resulet1)

tensor([[0.7581, 0.0244, 0.0510],
        [0.1640, 0.9836, 0.6480],
        [0.7777, 0.9896, 0.2816]])
tensor([[5.7581, 5.0244, 5.0510],
        [5.1640, 5.9836, 5.6480],
        [5.7777, 5.9896, 5.2816]])


In [54]:
# (3, 3)と(1, 3)の演算
b = torch.tensor([[5, 5, 5]])
print(b.shape, "\n")

result2 = a + b
print(a)
print(b)
print(result2)

torch.Size([1, 3]) 

tensor([[0.7581, 0.0244, 0.0510],
        [0.1640, 0.9836, 0.6480],
        [0.7777, 0.9896, 0.2816]])
tensor([[5, 5, 5]])
tensor([[5.7581, 5.0244, 5.0510],
        [5.1640, 5.9836, 5.6480],
        [5.7777, 5.9896, 5.2816]])


In [55]:
# (32, 128, 128, 3) と (128, 128, 3)の演算
c = torch.rand((32, 128, 128, 3))
d = torch.rand((128, 128, 3))

result3 = c + d
print(result3.shape)

torch.Size([32, 128, 128, 3])


In [56]:
# (32, 128, 128, 3) と (128, 128, 6)の演算は形状が不一致なためエラー
c = torch.rand((32, 128, 128, 3))
d = torch.rand((128, 128, 6))
# 以下はエラー
# result3 = c + d
# print(result3.shape)

In [57]:
# (1, 128, 128, 3)と(8, 128, 128, 1)の演算
e = torch.rand((1, 128, 128, 3))
f = torch.rand((8, 128, 128, 1))
result4 = e + f
print(result4.shape)

torch.Size([8, 128, 128, 3])
